In [132]:
import numpy as np
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
def distance(query, target):
    x_dist = (target[0] - query[0])**2
    y_dist = (target[1] - query[1])**2
    z_dist = (target[2] - query[2])**2
    dist = np.sqrt(x_dist+y_dist+z_dist)
    return dist

In [134]:
swarm_size = 20
dim = 4        # Dimension of X
epsilon = 1.0
options = {'c1': 1.5, 'c2':1.5, 'w':0.5}

constraints = (np.array([0,0,0,0]),
               np.array([np.pi,np.pi,np.pi,np.pi]))

d1 = 62
d2 = 102
d3 = 95
d4 = -35
d5 = 160 # 집계 끝을 기준으로 하였음

In [135]:
def rot_x(theta):
    T = np.array([[1, 0, 0, 0], 
                  [0, np.cos(theta), -np.sin(theta), 0],
                 [0, np.sin(theta), np.cos(theta), 0],
                 [0, 0, 0, 1]
                 ])
    return T

def rot_y(theta):
    T = np.array([[np.cos(theta), 0, np.sin(theta), 0], 
                  [0, 1, 0, 0],
                 [-np.sin(theta), 0, np.cos(theta), 0],
                 [0, 0, 0, 1]
                 ])
    return T

def rot_z(theta):
    T = np.array([[np.cos(theta), -np.sin(theta), 0, 0], 
                  [np.sin(theta), np.cos(theta), 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]
                 ])
    return T

def translation(d_x, d_y, d_z):
    T = np.array([[1, 0, 0, d_x],
                 [0, 1, 0, d_y],
                 [0, 0, 1, d_z],
                 [0, 0, 0, 1]])
    return T

In [136]:
def get_end_tip_position(params):
    t_00 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    t_01 = rot_z(params[0]) # 첫번째 모터 회전
    t_12 = translation(0, 0, d1)
    t_23 = rot_y(-np.pi/2)
    t_34 = rot_z(params[1]) # 두번째 모터 회전
    t_45 = translation(0, d2, 0)
    t_56 = rot_y(np.pi)
    t_67 = rot_z(params[2]) # 세번째 모터 회전
    t_78 = translation(0, d3, 0)
    t_89 = rot_z(params[3]) # 네번째 모터 회전
    t_910 = translation(d4, d5, 0)
    
    end_tip_m = t_00.dot(t_01).dot(t_12).dot(t_23).dot(t_34).dot(t_45).dot(t_56).dot(t_67).dot(t_78).dot(t_89).dot(t_910)
    
    pos = np.array([end_tip_m[0,3],end_tip_m[1,3],end_tip_m[2,3]])
    return pos

In [137]:
def opt_func(X):
    n_particles = X.shape[0]  # number of particles
    target = np.array([-141.29, -220,0]) # 각도를 찾을 점
    dist = [distance(get_end_tip_position(X[i]), target) for i in range(n_particles)]
    return np.array(dist)

In [138]:
%%time
# Call an instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                    dimensions=dim,
                                    options=options,
                                    bounds=constraints)

# Perform optimization
cost, joint_vars = optimizer.optimize(opt_func, iters=1000)

2022-05-12 21:52:29,682 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best: 100%|███████████████████████████████████████████████████████|1000/1000, best_cost=2.93e-14
2022-05-12 21:52:33,736 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 2.929642751054232e-14, best pos: [2.57070099 1.35095283 1.02788088 0.53388354]


CPU times: total: 4.19 s
Wall time: 4.06 s


In [139]:
print(get_end_tip_position(joint_vars))

[-1.41290000e+02 -2.20000000e+02  7.10542736e-15]


In [140]:
joint_vars

array([2.57070099, 1.35095283, 1.02788088, 0.53388354])

In [141]:
np.round_(np.degrees(joint_vars), 2)

array([147.29,  77.4 ,  58.89,  30.59])